In [5]:
import os
import pickle as pkl
import numpy as np

# plotting tools
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.palettes import Inferno, all_palettes
from bokeh.models import CustomJS, Slider, ColumnDataSource 
from bokeh.models import Whisker, HoverTool, Span, ColorBar
from bokeh.transform import linear_cmap, log_cmap, factor_cmap

from misc.database import Database

output_notebook()

Loading BokehJS ...

In [105]:
def feed_forward(w, x):
    return w*x

def rnn_1(x, t, v_0, lamda):
    return x/(1-lamda) + ((v_0-x)/(1-lamda))*np.exp(-(1-lamda)*t/15)

def integrator(v, x):
    return v+x/15

In [181]:
input_signal = np.ones(1000)*0
input_signal[200:203] = 1
# input_signal[300:500] = 1
# input_signal[600:700] = 1

plot_options = dict(width=550,
                    plot_height=250,
                    tools='pan,wheel_zoom,reset,save')

# for i in range(input_signal.shape[0]):
#     output.append(feed_forward(1, input_signal[i]))

lamda = [ -0.3, 0, 0.5, 0.6 ]
colors = [ 'red', 'green', 'blue', 'yellow' ]

output_plot = figure(title="Output Signal", **plot_options)

# for idx, l in enumerate(lamda):
cur_in = None
prev_in = None
cur_t = 0
output = []

for i in range(input_signal.shape[0]):
    if cur_in == None or input_signal[i] != cur_in:
        prev_in = cur_in
        cur_in = input_signal[i]
        cur_t = 0
    else:
        cur_t += 1

    if prev_in is not None:
#         print("i: {}, in: {}, rnn: {}".format(i, cur_in, rnn_1(cur_in, cur_t, prev_in)))
        output.append(min(rnn_1(cur_in, cur_t, output[-1], 1.2), 1e5))
#         print(output[-1])
#         output.append(integrator(output[-1], cur_in))
    else:
        output.append(0)

#     output = np.array(output)
input_plot = figure(title="Input Signal", **plot_options)

input_plot.line(range(input_signal.shape[0]),
                input_signal,
                line_width=2)
input_plot.xaxis.axis_label="time"
input_plot.yaxis.axis_label="signal strength"

output_plot.line(range(len(output)),
                output,
                line_width=2,
                line_color='blue')

# hline = Span(location=1, 
#              dimension='width', 
#              line_color='black',
#              line_dash='dotted',
#              line_width=1)
# output_plot.renderers.append(hline)
output_plot.xaxis.axis_label="time"
output_plot.yaxis.axis_label="strength"
show(input_plot)
show(output_plot)

In [129]:
x_list = np.arange(-5, 5, 0.01)
saturating = np.tanh(x_list)
saturating[saturating<0] = -1
saturating[saturating>=0] = 1

output_plot = figure(**plot_options)

output_plot.line(x_list[:int(saturating.shape[0]/2)],
                saturating[:int(saturating.shape[0]/2)],
                line_width=2,
                line_color='blue')

output_plot.line(x_list[int(saturating.shape[0]/2)+1:],
                saturating[int(saturating.shape[0]/2)+1:],
                line_width=2,
                line_color='blue')

# hline = Span(location=1, 
#              dimension='width', 
#              line_color='black',
#              line_dash='dotted',
#              line_width=1)
# output_plot.renderers.append(hline)
output_plot.xaxis.axis_label="input strength"
output_plot.yaxis.axis_label="output strength"
output_plot.xaxis.axis_label_text_font_size = "12pt"
output_plot.yaxis.axis_label_text_font_size = "12pt"
# show(input_plot)
show(output_plot)

In [144]:
x_list = np.arange(-5, 5, 0.01)
saturating = np.tanh(x_list)
saturating[saturating<0] = -1
saturating[saturating>=0] = 1

output_plot = figure(x_range=(-2, 2), y_range=(-2, 2), **plot_options)

output_plot.x([ 1, -1 ],
                   [ 1, -1 ],
                  size=20,
              color='red')
output_plot.circle([ 1, -1 ],
                   [ -1, 1 ],
                  size=20)

# hline = Span(location=1, 
#              dimension='width', 
#              line_color='black',
#              line_dash='dotted',
#              line_width=1)
# output_plot.renderers.append(hline)
output_plot.xaxis.visible = True
output_plot.xaxis.axis_label="v1 output"
output_plot.yaxis.axis_label="v2 output"
output_plot.xaxis.axis_label_text_font_size = "12pt"
output_plot.yaxis.axis_label_text_font_size = "12pt"
# show(input_plot)
show(output_plot)

In [167]:
M = np.array([[ 1, -1, -1 ],
              [ -1, 1, 1 ],
              [ -1, 1, 1 ]])

v_0 = np.array([ -1, -1, -1 ])

v_t = v_0
v_t = np.matmul(M, v_t)
print("v_t = ", v_t)
print(-0.5*np.dot(v_0, np.matmul(M, v_0)))

v_t =  [ 1 -1 -1]
-0.5


In [200]:
x_list = np.array(range(1000))
saturating = np.zeros(1000)
saturating[500:800] = np.exp((x_list[500:800]-500)/100)-1
saturating[800:1000] = saturating[799] + np.exp((x_list[500:700]-500)/100)

output_plot = figure(**plot_options)

output_plot.line(x_list,
                saturating,
                line_width=2)

# output_plot.line(x_list[int(saturating.shape[0]/2)+1:],
#                 saturating[int(saturating.shape[0]/2)+1:],
#                 line_width=2,
#                 line_color='blue')

# hline = Span(location=1, 
#              dimension='width', 
#              line_color='black',
#              line_dash='dotted',
#              line_width=1)
# output_plot.renderers.append(hline)
output_plot.xaxis.axis_label="input strength"
output_plot.yaxis.axis_label="output strength"
output_plot.xaxis.axis_label_text_font_size = "12pt"
output_plot.yaxis.axis_label_text_font_size = "12pt"
# show(input_plot)
show(output_plot)

In [241]:
def next_state(M, v_t):
    return np.matmul(M, v_t*np.exp(v_t))

v_0 = np.array([[1],
               [1]])
M = np.array([[-1.0, 0.5],
             [0.5, -1.0]])
v_t = v_0

out = []
for i in range(100):
    out.append(next_state(M, v_t))
#     print(v_t)
    v_t = out[-1]
    
output_plot = figure(**plot_options)

output_plot.line(range(100),
                [ val[0] for val in out ],
                line_color='red',
                line_width=2)
output_plot.line(range(100),
                [ val[1] for val in out ],
                line_width=2)

output_plot.xaxis.axis_label="time step"
output_plot.yaxis.axis_label="neuron output"
output_plot.xaxis.axis_label_text_font_size = "12pt"
output_plot.yaxis.axis_label_text_font_size = "12pt"
# show(input_plot)
show(output_plot)